# PGR: Plan de Gestion de Residuos
## Herramienta de Visualizacion y Analisis de Reportes

**Introducción**

En la actualidad, la sostenibilidad se ha vuelto una prioridad en el ámbito de la construcción. En este sentido, el reciclaje de los residuos generados es un punto de gran importancia. El ordenamiento y clasificación de residuos sólidos es una de las medidas más eficaces para lograr el desvío de vertederos y alcanzar altos porcentajes de reciclaje. Aún más eficaz, es una herramienta que permita recopilar y observar de forma cómoda los resultados de la gestión de residuos. 

Se plantea la necesidad de una herramienta que reciba las planillas confeccionadas en obra, las procese de forma sencilla y permita la visualización interactiva de los datos.

**Preguntas**

1. ¿Qué cantidad de residuos de generan en la obra en estudio?
2. ¿Cuál es la cantida de resiudos contabilizados por categoría?
3. ¿Cómo es la distribución de los resiudos al inicio y al final de la obra?
4. ¿Qué porcentaje de lo recolectado es reciclable?

**Metodología**

 - Lectura y procesamiento de los datos de planillas mediante Pandas.
 - Visualización de datos mediante Plotly Express, Graph Objects y Subplots.
 - Se exportan los gráficos en formato HTML.
 - Guardado de resultado en planilla de cálculo mediante Pandas.


**Caso de Estudio**

La planilla analizada pertenece a la obra NWT, cuya construcción inició en 2023 y finalizó en 2025.


### Proceso

**PASO 1:** Importar las bibliotecas necesarias para el programa



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px



**PASO 2:** Lectura de planilla de cálculo y almacenamiento de variables. Una dataframe por Hoja de la planilla.

Obs: Para la categoría "Suelos", el registro de hizo en unidades volumétricas (m3). Para igualar todas las cantidades en kilogramos, se aplica la densidad de 1100kg/m3.

In [2]:
pgr_metal = pd.DataFrame(pd.read_excel('Planilla_residuosA.xlsx',sheet_name='Metal',usecols=['FECHA','ORIGEN','UNIDAD','CANTIDAD']))
pgr_pet = pd.DataFrame(pd.read_excel('Planilla_residuosA.xlsx',sheet_name='Petreo',usecols=['FECHA','ORIGEN','UNIDAD','CANTIDAD']))
pgr_madera = pd.DataFrame(pd.read_excel('Planilla_residuosA.xlsx',sheet_name='Maderas',usecols=['FECHA','ORIGEN','UNIDAD','CANTIDAD']))
pgr_plast = pd.DataFrame(pd.read_excel('Planilla_residuosA.xlsx',sheet_name='Plasticos',usecols=['FECHA','ORIGEN','UNIDAD','CANTIDAD']))
pgr_nonrec = pd.DataFrame(pd.read_excel('Planilla_residuosA.xlsx',sheet_name='No Reciclable',usecols=['FECHA','ORIGEN','UNIDAD','CANTIDAD']))
pgr_papel = pd.DataFrame(pd.read_excel('Planilla_residuosA.xlsx',sheet_name='Papeles y Cartones',usecols=['FECHA','ORIGEN','UNIDAD','CANTIDAD']))
pgr_pelig = pd.DataFrame(pd.read_excel('Planilla_residuosA.xlsx',sheet_name='Peligrosos',usecols=['FECHA','ORIGEN','UNIDAD','CANTIDAD']))
pgr_suelo = pd.DataFrame(pd.read_excel('Planilla_residuosA.xlsx',sheet_name='Suelo',usecols=['FECHA','ORIGEN','UNIDAD','CANTIDAD']))
pgr_vidrio = pd.DataFrame(pd.read_excel('Planilla_residuosA.xlsx',sheet_name='Vidrio',usecols=['FECHA','ORIGEN','UNIDAD','CANTIDAD']))

fac_suelo = 1100 #Factor de conversión de m3 a kg de suelo.

pgr_suelo.loc[:,'CANTIDAD'] = pgr_suelo['CANTIDAD']*fac_suelo #De todos los registros de suelo, se selecciona la columna de Cantidad


**PASO 3.1:** Procesamiento de datos generales. 
- Cálculo de total general
- Cálculo de total por categoría
- Datos de años y nombre
- Categorías de estudio

Para el análisis del año inicial y final, es necesario que el índice tenga el formato datetime


In [3]:
pgr_total = pd.concat([pgr_metal,pgr_pet,pgr_madera,pgr_plast,pgr_nonrec,pgr_papel,pgr_pelig,pgr_suelo,pgr_vidrio]) #Se obtiene un único dataframe
datasets = [pgr_metal,pgr_pet,pgr_madera,pgr_plast,pgr_nonrec,pgr_papel,pgr_pelig,pgr_suelo,pgr_vidrio] # Se obtiene una lista de registros df

pgr_total['FECHA'] = pd.to_datetime(pgr_total['FECHA'],errors = 'coerce') #La columna de FECHA adquiere el formato datetime, para obtener el año del registro.
pgr_total['FECHA'] = pgr_total['FECHA'].fillna(pd.Timestamp('2024-01-01')) #Los registros con errores, se asignan a una fecha determinada

categorias = ['Metal','Petreos','Maderas','Plásticos','No reciclable','Papel y Cartón','Peligrosos','Suelo','Vidrio'] #Listado de los nombres de categorías

obra = pgr_metal.loc[0,'ORIGEN'] #Se obtiene el nombre de la obra.
min_año = pgr_total['FECHA'].dt.year.min() #El primer año de la obra.
max_año = pgr_total['FECHA'].dt.year.max() #El último año de la obra.


totales_pgr = [df['CANTIDAD'].sum() for df in datasets] #Lista de totales por categoría

bd = pd.DataFrame({'Categorias':categorias, 'Cantidad': totales_pgr})

suma_total = sum(totales_pgr) #Total KG de residuos registrados

C:\Users\HP\AppData\Local\Temp\ipykernel_2536\132048693.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pgr_total = pd.concat([pgr_metal,pgr_pet,pgr_madera,pgr_plast,pgr_nonrec,pgr_papel,pgr_pelig,pgr_suelo,pgr_vidrio]) #Se obtiene un único dataframe


**PASO 3.12:** Procesamiento de datos en los extremos temporales.

- Distribución de residuos en el primer año de la obra.
- Distribución de residuos en el último año de la obra.

In [4]:
for df in datasets:
    df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')

totales_pgr_min = [df.loc[df['FECHA'].dt.year==min_año, 'CANTIDAD'].sum() for df in datasets]
totales_pgr_max = [df.loc[df['FECHA'].dt.year==max_año, 'CANTIDAD'].sum() for df in datasets]





**PASO 4:** Análisis de materiales reciclables.

In [5]:
'''Elaboramos una lista que sume los materiales reciclables y los no reciclables
*Reciclables:
-Metal
-Pétreos
-Maderas
-Plásticos
-Papel y cartón
-Suelos
-Vidrios

*No Reciclables:
-Peligrosos
-No reciclables'''

categorias_rec = ['Metal','Petreos','Maderas','Plásticos','No reciclable','Papel y Cartón','Peligrosos','Suelo','Vidrio']
categorias_nonrec = ['No reciclable','Peligrosos']

rec_total = bd.loc[bd['Categorias'].isin(categorias_rec),'Cantidad'].sum() #Kilogramos de resiudos reciclables
nonrec_total = bd.loc[bd['Categorias'].isin(categorias_nonrec),'Cantidad'].sum()# Kilogramos de resiudos no reciclables


bd_rec = pd.DataFrame({'Tipo':['Reciclable','No Reciclable'],'Cantidad':[rec_total,nonrec_total]})

**PASO 5.1:** Visualización de Resultados: Totales

Para conocer la distribución, es ideal el gráfico de barras verticales. La gráfica permite seleccionar la categorías que se deseen visualizar en simultáneo.


In [6]:


fig1 = px.bar(
    bd,
    x='Categorias',
    y='Cantidad',
    color = 'Categorias',
    title = 'Total residuos recoletados en la obra',
    subtitle=f'Obra: {obra}({min_año}-{max_año})',
    labels = {'Cantidad': 'Recolectado (Kg)'},
    template='plotly_white',
    height=600
)

fig1.show()

***Guardar imagen***

In [7]:
fig1.write_html(f'Barras_total_{obra}.html', include_plotlyjs='cdn') #Se guarda en formato HTML.


**PASO 5.2:** Visualización de Resultados: Años extremos

Para conocer la distribución, es ideal el gráfico de barras verticales. La gráfica permite seleccionar la categorías que se deseen visualizar en simultáneo.
Se emplean subplots para visualizar ambos extremos temporales de forma simultánea.

In [8]:
from pickle import FALSE


fig2 = make_subplots(
    rows=1, cols=2,
    subplot_titles= (f'Año {min_año}',f'Año {max_año}')
    )

fig2.add_trace(
    go.Bar(
        x=categorias,
        y=totales_pgr_min,
        name = f'Año {min_año}',
        marker=dict(color=["#646ff8", '#ee543b', '#00cb96','#ab64f8','#ffa05b','#1bd2f2','#ff6691','#b5e781','#ff97fd']),
        ),
        row=1, col=1
)
fig2.add_trace(
    go.Bar(
        x=categorias,
        y=totales_pgr_max,
        name = f'Año {max_año}',
        marker=dict(color=["#646ff8", '#ee543b', '#00cb96','#ab64f8','#ffa05b','#1bd2f2','#ff6691','#b5e781','#ff97fd']),
        ),
        row=1, col=2
)

fig2.update_layout(
    template='plotly_white',
    height=500,
    title_text='Total residuos recoletados durante los extremos de la obra',
    title_subtitle_text = f'OBRA: {obra}',
    showlegend=False
)

fig2.show()

***Guardar imagen***

In [9]:
fig2.write_html(f'Barras_({min_año}-{max_año})_{obra}.html', include_plotlyjs='cdn') #Se guarda en formato HTML.

**PASO 5.3:** Visualización de Resultados: Porcentaje de Reciclaje.

Para este análisis, es preferible utilizar una gráfico de torta.

In [10]:
fig3 = px.pie(
    bd_rec,
    values='Cantidad',
    names='Tipo',
    labels='lala',
    title='Porcentaje de Recuperación de Residuos',
    subtitle=f'Obra: {obra}({min_año}-{max_año})',
    template='plotly_white',
    hole=0.5  # Donut chart (0 = pie completo, 0.5 = donut grande)
)

fig3.update_traces(
    textposition='outside',
    textinfo='percent+label',  # Muestra porcentaje y nombre
    hovertemplate='<b>%{label}</b><br>%{value:,.0f} Kg<br>%{percent}<extra></extra>',
    pull=[0.05, 0]
)

fig3.update_layout(
    uniformtext_minsize=12,
    margin=dict(t=50, b=50, l=50, r=50)
)
fig3.update

fig3.show()

***Guardar imagen***

In [11]:
fig3.write_html(f'Torta_{obra}.html', include_plotlyjs='cdn')


**PASO 6:** Ahorros ambientales resultantes.

Se tienen en cuenta los siguientes factores de conversión (ahorro):

- 1 Kg de metal reciclado ahorra  1.7 kWh de energía.
- 1 kg madera reciclada representan el ahorro de 1.9 kgCO2 equivalentes.
- 1 kg de plástico reciclado ahorran 39.26 litros de agua.
- 58.85 kg de Papel y Cartón representan 1 árbol de 10 años.

Estos valores serán comparados por los totales registrados anteriormente.

In [12]:
#Se crean las variables de conversion
fc_metal = 1.7
fc_madera = 1.9
fc_plastico = 39.26
fc_papel = 1/58.85

#categorias = ['Metal','Petreos','Maderas','Plásticos','No reciclable','Papel y Cartón','Peligrosos','Suelo','Vidrio']


energia = round(bd.loc[bd['Categorias']=='Metal','Cantidad'].iloc[0]*fc_metal,2)
co2 = round(bd.loc[bd['Categorias']=='Maderas','Cantidad'].iloc[0]*fc_madera,2)
agua = round(bd.loc[bd['Categorias']=='Plásticos','Cantidad'].iloc[0]*fc_plastico,2)
arbol = round(bd.loc[bd['Categorias']=='Papel y Cartón','Cantidad'].iloc[0]*fc_papel,2)




**PASO 7:** Informes Finales



Presentamos los resultados de forma breve, con los ahorros ambientales asociados a la gestión de residuos.

In [13]:
df1 = pd.DataFrame({
    'Categorías': categorias,
    f'Total obra {obra} (kg)':totales_pgr,
    }).set_index('Categorías') #Cantidades totales
df1.loc['Total'] = suma_total


df2 = pd.DataFrame(
    {'Categorías': categorias,
     'Año inicial (kg)':totales_pgr_min,
     'Año final (kg)': totales_pgr_max
    }).set_index('Categorías')

df3 = pd.DataFrame(
    {'Categorías': ['Metales','Madera','Plastico','Papel y Cartón'],
     'Ahorro relacionado':[energia,co2,agua,arbol],
     'Unidad': ['kWh','KgCO2eq','litros','árboles']
    }).set_index('Categorías')

with pd.ExcelWriter('Informe de PGR.xlsx') as writer:  

    df1.to_excel(writer, sheet_name='Totales')
    df2.to_excel(writer, sheet_name='Totales inicial y final')
    df3.to_excel(writer, sheet_name='Ahorro Ambiental')

**Análsis de Resultados**

Las preguntas planteadas al inicio, fueron respondidas por el código propuesto, para el caso de estudio presentado.

1. ¿Qué cantidad de residuos de generan en la obra en estudio?
2. ¿Cuál es la cantida de resiudos contabilizados por categoría?
3. ¿Cómo es la distribución de los resiudos al inicio y al final de la obra?
4. ¿Qué porcentaje de lo recolectado es reciclable?

Se presentan 3 gráficas en formato HTML para indicar totales, totales anuales y porcentaje de reciclaje, respectivamente.

Junto a esto, se genera una planilla con las informaciones mencionadas en cada pestaña.
